# DATA DOWNLOADING

In [ ]:
from src.constant import *
from src.configuration.configuration import Configuration

import os
from kaggle.api.kaggle_api_extended import KaggleApi

config_info = Configuration(config_file_path='/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/config/config.yaml')
#config_info.data_ingestion_config()
data_ingestion_config_info = config_info[DATA_INGESTION_CONFIG_KEY]
print(data_ingestion_config_info[DATA_INGESTION_KAGGLE_DATASET_URL_KEY])


api = KaggleApi()
api.authenticate()
dataset_url = data_ingestion_config_info[DATA_INGESTION_KAGGLE_DATASET_URL_KEY]
download_path = os.path.join(config_info[DATA_DIR], data_ingestion_config_info[DATA_INGESTION_ZIP_DATA_DIR_KEY])
api.dataset_download_files(dataset=dataset_url, path=download_path, unzip=False)

In [ ]:
os.getcwd()

'/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System'

In [ ]:
import os
os.chdir("/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/")

In [ ]:
from src.pipeline.pipeline import Pipeline

pipeline = Pipeline()
pipeline.start_data_ingestion()

# DATA EXTRACTION

In [19]:
zip_file_path = "/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/data/artifact/2024-01-28-15-24-50/data_ingestion/zip_data/imdb-movies-dataset.zip"
extract_path = os.getcwd()
note = "notebooks"
extract_path = os.path.join(extract_path, note)
import zipfile
with zipfile.ZipFile(zip_file_path,"r") as zip_ref:
    zip_ref.extractall(extract_path)

In [20]:
os.getcwd()

'/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System'

In [22]:
import os

directory_path = '/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/data/artifact/2024-01-29-16-00-34/data_ingestion/zip_data'

# Get the list of files in the directory
files = os.listdir(directory_path)[0]

# Print the filenames
print(files)


imdb-movies-dataset.zip


# Data Splitting

In [23]:
import pandas as pd

/var/folders/_3/2fm0mg0x5qdgpsw123gqb54r0000gp/T/ipykernel_74599/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
imdb_movies = "/Users/vambat1/Documents/Projects/Machine-Learning/04-Movie-Recommendation-System/data/artifact/2024-01-29-16-08-45/data_ingestion/raw_data/imdb_movies.csv"

In [25]:
df = pd.read_csv(imdb_movies)

In [26]:
df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [35]:
df["genre"].unique()

array(['Drama,\xa0Action', 'Science Fiction,\xa0Adventure,\xa0Action',
       'Animation,\xa0Adventure,\xa0Family,\xa0Fantasy,\xa0Comedy', ...,
       'Adventure,\xa0Comedy,\xa0Family,\xa0Science Fiction,\xa0Action',
       'Mystery,\xa0Thriller,\xa0Comedy',
       'Action,\xa0Adventure,\xa0Science Fiction,\xa0Thriller,\xa0Horror'],
      dtype=object)

In [84]:
df.columns

Index(['names', 'date_x', 'score', 'genre', 'overview', 'crew', 'orig_title',
       'status', 'orig_lang', 'budget_x', 'revenue', 'country'],
      dtype='object')

In [122]:
import pandas as pd
import numpy as np

df['score_cat'] = pd.cut(
    df["score"],
    bins = [0.0, 15.0, 30.0, 45.0, 60.0, 75.0, 90.0, 95.0, np.inf],
    labels = [1, 2, 3, 4, 5, 6, 7, 8]
)

In [127]:
df.score_cat.value_counts()

score_cat
5    5932
4    2509
6    1153
3     270
2      60
1      29
8      10
7       4
Name: count, dtype: int64

In [132]:
df['score_cat'] = df['score_cat'].astype(float)
df['score_cat'] = df['score_cat'].fillna(df.score_cat.median())

In [133]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df['score_cat']):
    strat_train_set = df.loc[train_index].drop(['score_cat'], axis=1)
    start_test_set = df.loc[test_index].drop(['score_cat'], axis=1)

In [138]:
start_test_set.score.unique()

array([ 61.,  74.,  59.,  63.,  51.,  71.,  76.,  69.,  58.,  48.,  68.,
        67.,  66.,  80.,  60.,  72.,  44.,  75.,  53.,  54.,  77.,  73.,
        62.,  47.,  55.,  57.,  64.,  65.,  40.,  46.,  26.,  70.,  49.,
        78.,   0.,  29.,  81.,  90.,  56.,  91.,  83.,  50.,  79.,  52.,
        45.,  43.,  20.,  15.,  82.,  85.,  42.,  36.,  87.,  37.,  38.,
        10.,  30., 100.,  84.,  41.,  35.,  39.,  34.,  23.,  86.,  25.,
        33.])

In [139]:
strat_train_set.score.unique()

array([ 63.,  75.,  72.,  76.,  65.,  66.,  70.,  71.,   0.,  81.,  73.,
        77.,  68.,  69.,  41.,  83.,  67.,  46.,  74.,  78.,  51.,  55.,
        57.,  58.,  43.,  64.,  79.,  53.,  40.,  62.,  50.,  59.,  47.,
        10.,  54.,  60.,  87.,  80.,  61.,  35.,  56.,  82.,  52.,  90.,
        44.,  36.,  49.,  85.,  45.,  20.,  84.,  39.,  48., 100.,  86.,
        37.,  25.,  38.,  42.,  30.,  13.,  92.,  32.,  33.,  95.,  22.,
        18.,  34.,  28.,  17.,  15.,  98.,  16.,  27.,  93.])

211

In [95]:
df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US
